In [1]:
# STEP 2: Import libraries
import spacy
from transformers import pipeline

# Load spacy for sentence splitting
nlp = spacy.load("en_core_web_sm")

c:\Users\Μαρία\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# STEP 3: Load pipelines
grammar_correction = pipeline("text2text-generation", model="vennify/t5-base-grammar-correction")
paraphraser = pipeline("text2text-generation", model="Vamsi/T5_Paraphrase_Paws")



Device set to use cpu
c:\Users\Μαρία\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Μαρία\.cache\huggingface\hub\models--Vamsi--T5_Paraphrase_Paws. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Error while downloading from https://cdn-lfs.hf.co/Vamsi/T5_Paraphr

In [ ]:
# STEP 4: Define helper functions
def split_to_sentences(text):
    doc = nlp(text)
    return [sent.text.strip() for sent in doc.sents]

def grammar_fix(sentence):
    input_text = f"grammar: {sentence}"
    output = grammar_correction(input_text, max_new_tokens=128, do_sample=False)
    return output[0]["generated_text"]

def paraphrase(sentence):
    input_text = f"paraphrase: {sentence} </s>"
    output = paraphraser(input_text, max_new_tokens=128, do_sample=True, num_return_sequences=1)
    return output[0]["generated_text"]


In [ ]:
# STEP 5: Pipeline execution
def process_text(text):
    sentences = split_to_sentences(text)
    for i, sentence in enumerate(sentences, 1):
        fixed = grammar_fix(sentence)
        para = paraphrase(fixed)
        print(f"🔹 Sentence {i}")
        print(f"Original     : {sentence}")
        print(f"Grammar fixed: {fixed}")
        print(f"Paraphrased  : {para}")
        print("-" * 70)


In [ ]:
# STEP 6: Try it out
input_text = """
During our final discuss, I told him about the new submission — the one we were waiting since
last autumn, but the updates was confusing as it not included the full feedback from reviewer or
maybe editor?
Anyway, I believe the team, although bit delay and less communication at recent days, they really
tried best for paper and cooperation. We should be grateful, I mean all of us, for the acceptance
and efforts until the Springer link came finally last week, I think.
Also, kindly remind me please, if the doctor still plan for the acknowledgments section edit before
he sending again. Because I didn’t see that part final yet, or maybe I missed, I apologize if so.
Overall, let us make sure all are safe and celebrate the outcome with strong coffee and future
targets
"""

process_text(input_text)


🔹 Sentence 1
Original     : During our final discuss, I told him about the new submission — the one we were waiting since
last autumn, but the updates was confusing as it not included the full feedback from reviewer or
maybe editor?
Grammar fixed: During our final discussion, I told him about the new submission — the one we were waiting for since last autumn, but the updates were confusing as it did not include the full feedback from reviewer or maybe editor?
Paraphrased  : During our final discussion, I told him about the new submission — the one we had been waiting for since last autumn , but the updates were confusing as it did not include the full feedback from the reviewer or maybe the editor?
----------------------------------------------------------------------
🔹 Sentence 2
Original     : Anyway, I believe the team, although bit delay and less communication at recent days, they really
tried best for paper and cooperation.
Grammar fixed: Anyway, I believe the team, although a bit